<a href="https://colab.research.google.com/github/Nzau1/DR.-DIAGNOSIS/blob/main/disease_diagnosis_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define file paths
data_folder = "/content/drive/MyDrive/diagnosis/processed_data"
X_train_path = f"{data_folder}/X_train.csv"
X_test_path = f"{data_folder}/X_test.csv"
y_train_path = f"{data_folder}/y_train.csv"
y_test_path = f"{data_folder}/y_test.csv"

# Load preprocessed data
X_train = pd.read_csv(X_train_path)
X_test = pd.read_csv(X_test_path)
y_train = pd.read_csv(y_train_path)
y_test = pd.read_csv(y_test_path)

# Encode target labels (if not already encoded)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train.values.ravel())  # Fit on training data

# Ensure test labels are only those seen during training
y_test = y_test.values.ravel()
y_test = np.array([label_encoder.transform([label])[0] if label in label_encoder.classes_ else -1 for label in y_test])

# Remove invalid test samples
valid_indices = y_test != -1
X_test = X_test[valid_indices]
y_test = y_test[valid_indices]

# Convert to TensorFlow format
X_train, X_test = np.array(X_train), np.array(X_test)

# Define model architecture
def create_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        BatchNormalization(),
        Dropout(dropout_rate),

        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(dropout_rate),

        Dense(len(np.unique(y_train)), activation='softmax')  # Multi-class classification
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter tuning
best_lr = 0.001  # Default learning rate
best_dropout = 0.3  # Default dropout

# Train model
model = create_model(learning_rate=best_lr, dropout_rate=best_dropout)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Save model to Google Drive
model_path = "/content/drive/MyDrive/diagnosis/disease_dnn_model.h5"
model.save(model_path)
print(f"\n✅ Model saved to {model_path}!")

# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\n🔍 Test Accuracy: {test_acc * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 32s 4ms/step - accuracy: 0.2995 - loss: 3.6624 - val_accuracy: 0.7762 - val_loss: 0.7491
Epoch 2/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - accuracy: 0.5980 - loss: 1.4777 - val_accuracy: 0.8124 - val_loss: 0.5891
Epoch 3/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.6378 - loss: 1.3023 - val_accuracy: 0.8234 - val_loss: 0.5314
Epoch 4/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 42s 4ms/step - accuracy: 0.6595 - loss: 1.2170 - val_accuracy: 0.8219 - val_loss: 0.5096
Epoch 5/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - accuracy: 0.6704 - loss: 1.1694 - val_accuracy: 0.8324 - val_loss: 0.4948
Epoch 6/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.6811 - loss: 1.1253 - val_accuracy: 0.8350 - val_loss: 0.4764
Epoch 7/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.6863 - loss: 1.1039 - val_accuracy: 0.8361 - val_loss: 0.4690
Epoch 8/50
6174/6174 ━━━━━━━━━━━━━━━━━━━━ 43s 4ms/step - accuracy: 0.6918 - loss: 1


✅ Model saved to /content/drive/MyDrive/diagnosis/disease_dnn_model.h5!
1544/1544 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8513 - loss: 0.4070

🔍 Test Accuracy: 84.81%
